### 0. Helper functions

In [34]:
import numpy as np

class bin_data:
    def __init__(self, env):
        self.total_capacity = np.full(env.num_bins, 0) # Total capacity seen across all trials
        self.capacity_used = np.full(env.num_bins, 0) # Total capacity used across all trials
        self.steps_taken = 0 # Number of timesteps taken before trial terminated
        self.placed = 0 # Number of items correctly placed
        self.misplaced = 0
        self.discarded = 0
        self.data_points = 0 # Number of trials we've witnessed
    

    def log(self, env):
        self.total_capacity = np.add(self.total_capacity, env.capacity)
        self.capacity_used = np.add(self.capacity_used, env.state[:-1])
        self.steps_taken += sum(env.logs.values())
        self.placed += env.logs["placed"]
        self.misplaced += env.logs["misplaced"]
        self.discarded += env.logs["discarded"]
        self.data_points += 1
        

    def get_avg(self):
        percentages = 100 - ((self.capacity_used/self.total_capacity) * 100)
        
        return {
            "steps" : self.steps_taken / self.data_points,
            "utilization" : sum(percentages) / len(percentages),
            "accuracy" : (self.placed / (self.placed + self.misplaced)) * 100
        }
    

    def print_data(self):
        data = self.get_avg()
        print(f'Average number of steps taken: {data["steps"]}')
        print(f'Average bin utilization: {round(data["utilization"], 2)}%')
        print(f'Accuracy: {round(data["accuracy"], 2)}%')

### 1. Import dependencies

In [ ]:
!pip install tensorflow==2.7.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [2]:
from BinPackingEnvironment1D import BinPacking

### 2. Create environment

In [3]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [4]:
env.action_space.sample()

2

In [5]:
env.observation_space.sample()

array([10,  4,  8, 19,  1, 10, 15, 15, 18,  2,  3])

### 3. Run baseline test (No ML)

In [41]:
control_data = bin_data(env)

In [42]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    control_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-3755
{'placed': 45, 'misplaced': 376, 'discarded': 40}
Episode:2 Score:-4350
{'placed': 47, 'misplaced': 435, 'discarded': 47}
Episode:3 Score:-3500
{'placed': 44, 'misplaced': 350, 'discarded': 44}
Episode:4 Score:-3300
{'placed': 45, 'misplaced': 331, 'discarded': 35}
Episode:5 Score:-1717
{'placed': 43, 'misplaced': 173, 'discarded': 30}
Episode:6 Score:-4466
{'placed': 48, 'misplaced': 446, 'discarded': 54}
Episode:7 Score:-4222
{'placed': 47, 'misplaced': 423, 'discarded': 39}
Episode:8 Score:-3002
{'placed': 49, 'misplaced': 302, 'discarded': 31}
Episode:9 Score:-3428
{'placed': 45, 'misplaced': 344, 'discarded': 33}
Episode:10 Score:-4869
{'placed': 51, 'misplaced': 487, 'discarded': 50}


### 4. Train an RL Model

In [8]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [9]:
# Will throw an error if these don't exist
log_path = os.path.join('Training', 'Logs')

In [10]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [11]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=80000)

Logging to Training\Logs\PPO_54
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 494       |
|    ep_rew_mean     | -4.01e+03 |
| time/              |           |
|    fps             | 270       |
|    iterations      | 1         |
|    time_elapsed    | 7         |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 372         |
|    ep_rew_mean          | -2.86e+03   |
| time/                   |             |
|    fps                  | 364         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011663858 |
|    clip_fraction        | 0.0745      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.39       |
|    explained_variance   | -3

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 281          |
|    ep_rew_mean          | -1.49e+03    |
| time/                   |              |
|    fps                  | 558          |
|    iterations           | 11           |
|    time_elapsed         | 40           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0114904605 |
|    clip_fraction        | 0.14         |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.55        |
|    explained_variance   | 4.82e-05     |
|    learning_rate        | 0.0003       |
|    loss                 | 1.09e+03     |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.0241      |
|    value_loss           | 2.19e+03     |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 313          |
|    ep_rew_mean          | -747         |
| time/                   |              |
|    fps                  | 576          |
|    iterations           | 21           |
|    time_elapsed         | 74           |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0054755853 |
|    clip_fraction        | 0.0564       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.865       |
|    explained_variance   | 0.702        |
|    learning_rate        | 0.0003       |
|    loss                 | 70.3         |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.0148      |
|    value_loss           | 118          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 325          |
|    ep_rew_mean          | -300         |
| time/                   |              |
|    fps                  | 573          |
|    iterations           | 31           |
|    time_elapsed         | 110          |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0082489755 |
|    clip_fraction        | 0.087        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.755       |
|    explained_variance   | 0.795        |
|    learning_rate        | 0.0003       |
|    loss                 | 18.1         |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.0127      |
|    value_loss           | 44.5         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

### 5. Save model

In [ ]:
#PPO_Path = os.path.join('Training', 'Saved Models', 'Constant_PPO_Model_Discard_Penalty')

In [ ]:
#model.save(PPO_Path)

### 6. Load model

In [ ]:
#model = PPO.load(PPO_Path, env=env)

### 7. Test model

In [58]:
real_data = bin_data(env)
env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

In [59]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    obs = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    real_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-92
{'placed': 78, 'misplaced': 0, 'discarded': 170}
Episode:2 Score:-3
{'placed': 57, 'misplaced': 1, 'discarded': 50}
Episode:3 Score:-15
{'placed': 75, 'misplaced': 0, 'discarded': 90}
Episode:4 Score:-34
{'placed': 59, 'misplaced': 2, 'discarded': 73}
Episode:5 Score:-59
{'placed': 65, 'misplaced': 3, 'discarded': 94}
Episode:6 Score:-39
{'placed': 60, 'misplaced': 1, 'discarded': 89}
Episode:7 Score:-42
{'placed': 69, 'misplaced': 2, 'discarded': 91}
Episode:8 Score:-77
{'placed': 67, 'misplaced': 4, 'discarded': 104}
Episode:9 Score:-67
{'placed': 63, 'misplaced': 2, 'discarded': 110}
Episode:10 Score:-76
{'placed': 65, 'misplaced': 4, 'discarded': 101}


In [55]:
control_data.print_data()

Average number of steps taken: 453.4
Average bin utilization: 98.85%
Accuracy: 11.23%


In [60]:
real_data.print_data()

Average number of steps taken: 164.9
Average bin utilization: 97.35%
Accuracy: 97.19%
